# POC Model

__Install packages__

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import glob
from datetime import datetime
import statistics
import math

In [14]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords
import glob
from datetime import datetime
import statistics 

//anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
//anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


### Data Sources Integration

In [2]:
data = []
for filename in glob.glob('*.csv'):
    data.append(filename)
data

['COPA_forum_threads.csv',
 'Parsed Magazine.csv',
 'wiki_formal_total_clean.csv',
 'COPA_Blogs.csv']

In [10]:
forum = pd.read_csv(data[0])
data = forum.FormattedBody.values.tolist()

In [16]:
# Create a model for topic classification based on the dataset we have 
# (It may cause some problems because the model will be applied to the dataset later to give each content a topic.)
# (The problem can be solved when we get the entire dataset since we can take part of the contents as corpus.)

# Get some stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [18]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [19]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [20]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           passes=10)

pprint(lda_model.print_topics(num_words = 5))
# Get 20 related topics
doc_lda = lda_model[corpus]

[(0,
  '0.035*"copa" + 0.031*"member" + 0.028*"event" + 0.024*"email" + '
  '0.014*"register"'),
 (1,
  '0.045*"hour" + 0.040*"engine" + 0.030*"annual" + 0.028*"oil" + '
  '0.026*"replace"'),
 (2,
  '0.049*"system" + 0.036*"traffic" + 0.036*"avionic" + 0.035*"display" + '
  '0.033*"upgrade"'),
 (3,
  '0.042*"fly" + 0.027*"flight" + 0.020*"trip" + 0.018*"weather" + '
  '0.017*"plan"'),
 (4,
  '0.036*"update" + 0.018*"datum" + 0.017*"use" + 0.017*"page" + 0.015*"card"'),
 (5,
  '0.096*"pilot" + 0.050*"flight" + 0.043*"training" + 0.027*"aircraft" + '
  '0.019*"cirrus"'),
 (6,
  '0.219*"com" + 0.069*"video" + 0.043*"watch" + 0.043*"article" + '
  '0.042*"http_www"'),
 (7, '0.028*"get" + 0.026*"go" + 0.021*"plane" + 0.019*"say" + 0.016*"think"'),
 (8,
  '0.112*"post" + 0.043*"thread" + 0.039*"find" + 0.033*"forum" + 0.029*"see"'),
 (9,
  '0.044*"would" + 0.031*"thank" + 0.030*"look" + 0.025*"plane" + 0.022*"buy"'),
 (10,
  '0.035*"approach" + 0.020*"fly" + 0.018*"runway" + 0.015*"land" + '

In [5]:
wiki = pd.read_csv(data[2])
wiki = wiki[['Date', 'Title', 'Content', 'Author','Like']]
wiki['Author'] = [a.strip() for a in wiki['Author']]
wiki['Comment'] = 0
wiki['Resource'] = 'Wiki'

In [6]:
magazine = pd.read_csv(data[1])
magazine_date_list = list(magazine['Magazine'].unique())
# Get the month of magazine from its name
# Since the the COPA magazines are released monthly, so it only has year and month
# in its date column. In this case we just assume all magazines were released on the 
# first day of that month.
magazine_date_dict = {magazine_date_list[0]: '2015-01-01', magazine_date_list[1]: '2019-01-01',
                      magazine_date_list[2]: '2015-07-01', magazine_date_list[3]: '2019-04-01',
                      magazine_date_list[4]: '2019-06-01', magazine_date_list[5]: '2018-01-01',
                      magazine_date_list[6]: '2017-09-01', magazine_date_list[7]: '2016-09-01',
                      magazine_date_list[8]: '2017-11-01', magazine_date_list[9]: '2015-05-01',
                      magazine_date_list[10]: '2015-09-01', magazine_date_list[11]: '2006-09-01', 
                      magazine_date_list[12]: '2018-06-01', magazine_date_list[13]: '2019-07-01',
                      magazine_date_list[14]: '2017-04-01', magazine_date_list[15]: '2016-04-01',
                      magazine_date_list[16]: '2018-11-01', magazine_date_list[17]: '2015-09-01',
                      magazine_date_list[18]: '2019-03-01', magazine_date_list[19]: '2006-11-01',
                      magazine_date_list[20]: '2016-01-01', magazine_date_list[21]: '2019-05-01',
                      magazine_date_list[22]: '2016-03-01', magazine_date_list[23]: '2016-05-01',
                      magazine_date_list[24]: '2006-07-01', magazine_date_list[25]: '2017-06-01',
                      magazine_date_list[26]: '2018-07-01', magazine_date_list[27]: '2018-09-01',
                      magazine_date_list[28]: '2017-01-01', magazine_date_list[29]: '2012-11-01'}
magazine['Magazine'] = magazine['Magazine'].map(magazine_date_dict)
magazine.columns = ['Date', 'Title', 'Content', 'Author']
magazine['Like'] = 0
magazine['Comment'] = 0
magazine['Resource'] = 'Magazine'

In [7]:
blog = pd.read_csv(data[3], encoding = 'unicode_escape')
blog = blog[['Date', 'Title', 'Body', 'Author', 'Like','Comment']]
blog.columns = ['Date', 'Title', 'Content', 'Author', 'Like','Comment']
blog['Date'] = [d.split(' ')[0].split('/')[2] + '-' + d.split(' ')[0].split('/')[0] for d in blog['Date']]
blog['Date'] = blog['Date'].map(lambda x: x + '-01')
blog['Resource'] = 'Blog'

In [6]:
pdList = [magazine, wiki, blog]
# Combine three datasets together
df = pd.concat(pdList)
df.reset_index(drop = True, inplace = True)

In [7]:
#Ranking the source, blog is the most important source so have 3
raw_data = {'Resource': ['Wiki','Blog','Magazine'], 'SourceScore': [1, 2, 3]}
Source_tb = pd.DataFrame(raw_data, columns = ['Resource', 'SourceScore'])
#add source score to the main table
df = pd.merge(df, Source_tb, left_on = 'Resource', right_on = 'Resource')

# Generate Attributes

__Generate Recency Score__

In [8]:
def  Calculate_RecencyScore(date):
    '''
    Recency Rate = log( 1 + 1/(days between the post date and current date + 1))
    '''
    date_datetime = datetime.strptime(date, '%Y-%m-%d').date()
    rececncy_rate = math.log10(1/(((datetime.date(datetime.now()))-date_datetime).days+1)+1)
    return rececncy_rate
    

df['RecencyRate'] = df['Date'].map(lambda y: Calculate_RecencyScore(y))

__Generate Author Score__

In [9]:
def Calculate_AuthorScore(author):
    '''
    AuthorScore = Number of posts of specified content source * pre-determined weight
    '''
    author_score = df[df['Author'] == author]['SourceScore'].sum()
    return author_score

author_list = list(df['Author'].unique())
df['AuthorScore'] = df['Author'].map(lambda x: Calculate_AuthorScore(x), author_list)

__Normalization__

In [10]:
def Normalization(col):
    nomolized_col = (df[col] - df[col].mean()) / df.loc[:, col].std()
    return nomolized_col

df['RecencyRate'] = Normalization('RecencyRate')
df['AuthorScore'] = Normalization('AuthorScore')

In [11]:
df.head()

,Date,Title,Content,Author,Like,Comment,Resource,SourceScore,RecencyRate,AuthorScore
0,2015-01-01,President's Column,JANUARY FEBRUARY 20154CIRRUS PILOTAs this is b...,NaN,0,0,Magazine,3,-0.465959,-1.209951
1,2015-01-01,COPA News,JANUARY FEBRUARY 20156CIRRUS PILOTWith this is...,NaN,0,0,Magazine,3,-0.465959,-1.209951
2,2015-01-01,Regional News,JANUARY FEBRUARY 201512CIRRUS PILOTby GIL WILL...,GIL WILLIAMSON,0,0,Magazine,3,-0.465959,0.211050
3,2015-01-01,Cirrus Perspective,JANUARY FEBRUARY 201518CIRRUS PILOTIts hard to...,NaN,0,0,Magazine,3,-0.465959,-1.209951
4,2015-01-01,Member Spotlight,JANUARY FEBRUARY 201522CIRRUS PILOTCirrus Pilo...,KIM BLONIGEN,0,0,Magazine,3,-0.465959,0.211050
